In [ ]:
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', sys.maxsize)
pd.set_option('display.max_colwidth', None)

In [ ]:
with open("UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml", "r") as file:
    handler20 = file.read()

with open("UC 3/etl_generated/R21-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml", "r") as file:
    handler21 = file.read()

soup20 = bs(handler20, "lxml-xml")
list_of_all_ids20 = soup20.find_all("SHORT-NAME")
soup21 = bs(handler21, "lxml-xml")
list_of_all_ids21 = soup21.find_all("SHORT-NAME")

list_of_all_ids20 = [id.getText() for id in list_of_all_ids20]
list_of_all_ids21 = [id.getText() for id in list_of_all_ids21]

df20 = pd.DataFrame({"ids20": list_of_all_ids20})
df21 = pd.DataFrame({"ids21": list_of_all_ids21})
df_merged = pd.merge(df20, df21, how = "outer", left_on = "ids20", right_on = "ids21")
df_merged["type"] = "unchanged"
df_merged["type"][df_merged.ids21.isna()] = "delete"
df_merged["type"][df_merged.ids20.isna()] = "new"
df_merged.head()

# New Part: Change Detection

In [ ]:
import xml.etree.ElementTree as ET
from lxml import etree

In [ ]:
tree = etree.parse('UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml')
ids20simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]/x:TRACE/x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})
text20simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]//x:L-1/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [ ]:
text20simple

In [ ]:
data20simple = pd.DataFrame({"ids20": ids20simple, "text20": text20simple})
data20simple = data20simple.loc[data20simple["ids20"].str[:4] != "ECUC"]

In [ ]:
data20simple

In [ ]:
tree = etree.parse('UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml')
ids21simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]/x:TRACE/x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})
text21simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]//x:L-1/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [ ]:
data21simple = pd.DataFrame({"ids21": ids21simple, "text21": text21simple})
data21simple = data21simple.loc[data21simple["ids21"].str[:4] != "ECUC"]

In [ ]:
data21simple

In [ ]:
datamergedsimple = pd.merge(data20simple, data21simple, how="outer", left_on="ids20", right_on="ids21")

In [ ]:
datamergedsimple["type"] = "unchanged"
datamergedsimple["type"].loc[datamergedsimple.ids21.isna()] = "delete"
datamergedsimple["type"].loc[datamergedsimple.ids20.isna()] = "new"

In [ ]:
datamergedsimple.loc[0, "text21"] = "asdf"

In [ ]:
datamergedsimple.loc[(datamergedsimple["type"] == "unchanged") & (datamergedsimple["text20"] != datamergedsimple["text21"]), "type"] = "changed"

In [ ]:
datamergedsimple.loc[datamergedsimple["ids20"] == "SWS_CANIF_00378"]

## BS4 Table

In [ ]:
with open("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerA = file.read()

with open("UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerB = file.read()

soupA = bs(handlerA, "lxml-xml")
list_of_all_idsA = soupA.find_all("SHORT-NAME")
soupB = bs(handlerB, "lxml-xml")
list_of_all_idsB = soupB.find_all("SHORT-NAME")

list_of_all_idsA = [id.getText() for id in list_of_all_idsA]
list_of_all_idsB = [id.getText() for id in list_of_all_idsB]

dfA = pd.DataFrame({"idsA": list_of_all_idsA})
dfB = pd.DataFrame({"idsB": list_of_all_idsB})
df_merged = pd.merge(dfA, dfB, how = "outer", left_on = "idsA", right_on = "idsB")
df_merged["type"] = "unchanged"
df_merged["type"][df_merged.idsB.isna()] = "deleted"
df_merged["type"][df_merged.idsA.isna()] = "new"

df = df_merged.copy()

for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][0])) and (not pd.isna(row[1][0])):
        df['Text Release A'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][0]).parent.find_all('L-1')]    

soup = soupB
for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][1])) and (not pd.isna(row[1][1])):
        df['Text Release B'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][1]).parent.find_all('L-1')]


## Same table with xpath

In [ ]:
treeA = etree.parse("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml")
treeB = etree.parse("UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml")

idsA = treeA.xpath(f"//x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})
idsB = treeB.xpath(f"//x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})

dfA = pd.DataFrame({"idsA": idsA})
dfB = pd.DataFrame({"idsB": idsB})
df = pd.merge(dfA, dfB, how = "outer", left_on = "idsA", right_on = "idsB")
df["type"] = "unchanged"
df["type"][df_merged.idsB.isna()] = "deleted"
df["type"][df_merged.idsA.isna()] = "new"

textA = []
structureA = []
for id in df["idsA"]:
    textA.append(treeA.xpath(f"//x:SHORT-NAME[./text() = '{id}']/..//x:L-1//text()", namespaces={'x': 'http://autosar.org/schema/r4.0'}))
    structureA.append([elem.tag[32:] for elem in treeA.xpath(f"//x:SHORT-NAME[./text() = '{id}']/../..//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})])

textB = []
structureB = []
for id in df["idsB"]:
    textB.append(treeB.xpath(f"//x:SHORT-NAME[./text() = '{id}']/..//x:L-1//text()", namespaces={'x': 'http://autosar.org/schema/r4.0'}))
    structureB.append([elem.tag[32:] for elem in treeB.xpath(f"//x:SHORT-NAME[./text() = '{id}']/../..//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})])

df["Text Release A"] = textA
df["Text Release B"] = textB

df["structureA"] = structureA
df["structureB"] = structureB


In [ ]:
df.loc[df["idsA"].str.contains("ECUC", na = False)]

In [ ]:
# Add Structure to compare table
# df["structureA"] = [getStructure("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", id) for id in df["idsA"]]
# df["structureB"] = [getStructure("UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", id) for id in df["idsB"]]

In [ ]:
df.loc[df["structureA"] != df["structureB"]]

# Get different tag formats

In [ ]:
def getStructure(tablepath, id):
    tree = etree.parse(tablepath)
    return [elem.tag[32:] for elem in tree.xpath(f"//x:SHORT-NAME[./text() = '{id}']/../..//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})]

In [ ]:
getStructure("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", 'SWS_CANIF_00378')

In [ ]:
tree = etree.parse("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml")
[elem.tag[32:] for elem in tree.xpath(f"//x:SHORT-NAME[./text() = SWS_CANIF_00378]/../..//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})]


In [ ]:
tree.xpath(f"//x:SHORT-NAME[./text() = 'SWS_CANIF_00378']/../..//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [ ]:
[elem for elem in tree.iter() if elem.tag == "{http://autosar.org/schema/r4.0}TRACE"]

In [ ]:
for elem in tree.iter():
    if elem.tag == "{http://autosar.org/schema/r4.0}TRACE":
        print(elem.tag)

In [ ]:
tree.findall("{http://autosar.org/schema/r4.0}REQUIREMENTS")

In [ ]:
def getStructures(tablepath):
    tree = etree.parse(tablepath)

    ls = []
    struct = []
    for tag in [elem.tag[32:] for elem in tree.xpath("//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})][1:]:
        if tag == "REQUIREMENT":
            ls.append(struct)
            struct = [tag]
        elif tag != "REQUIREMENT": 
                struct.append(tag)
    ls = ls[1:]

    set = []
    for x in ls:
        if x not in set:
            set.append(x)

    counts = []
    for struct in set:
        counts.append(ls.count(struct))

    rel = []
    for count in counts:
        rel.append(count/sum(counts))

    structtable = pd.DataFrame({"structure": set, "count": counts, "relative": rel})

    return structtable

In [ ]:
s20 = getStructures('UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s20.loc[s20.relative > 0.15]

In [ ]:
s21 = getStructures('UC 3/etl_generated/R21-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s21.loc[s21.relative > 0.15]

In [ ]:
s22 = getStructures('UC 3/etl_generated/R22-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s22.loc[s22.relative > 0.15]

In [ ]:
s20 = getStructures('UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')

# Filter StructureTable by number of L-1 tags
structs = []
counts = []
relatives = []
for index, row in s20.iterrows():
    if row.structure.count("L-1") == 1:
        structs.append(row.structure)
        counts.append(row[1])
        relatives.append(row.relative)
df = pd.DataFrame({"structure": structs, "count": counts, "relative": relatives})

In [ ]:
sum(df.relative)

In [ ]:
df

In [ ]:
s20